In [2]:
import numpy as np
import pandas as pd

In [3]:
def Entropy(values):
    normalized= values/np.sum(values)
    normalized[normalized==0]= 1
    return -1.0* np.dot(normalized.T, np.log2(normalized))
    

In [4]:
def Information_gain(S,A):
    entropies= np.array([entropy(A[e,:]) for e in range(A.shape[0])])

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

train_set = pd.read_csv(
    'https://raw.githubusercontent.com/sjwhitworth/golearn/master/examples/datasets/tennis.csv')

x = train_set.iloc[:, :4].values
y = train_set.iloc[:, 4].values
print(x, y)

[['sunny' 'hot' 'high' False]
 ['sunny' 'hot' 'high' True]
 ['overcast' 'hot' 'high' False]
 ['rainy' 'mild' 'high' False]
 ['rainy' 'cool' 'normal' False]
 ['rainy' 'cool' 'normal' True]
 ['overcast' 'cool' 'normal' True]
 ['sunny' 'mild' 'high' False]
 ['sunny' 'cool' 'normal' False]
 ['rainy' 'mild' 'normal' False]
 ['sunny' 'mild' 'normal' True]
 ['overcast' 'mild' 'high' True]
 ['overcast' 'hot' 'normal' False]
 ['rainy' 'mild' 'high' True]] ['no' 'no' 'yes' 'yes' 'yes' 'no' 'yes' 'no' 'yes' 'yes' 'yes' 'yes' 'yes'
 'no']


In [6]:
fields = list(train_set.columns.values)
print(fields)

['outlook', 'temp', 'humidity', 'windy', 'play']


In [7]:
def entropy(s):
    res = 0
    val, counts = np.unique(s, return_counts=True)
    freqs = counts.astype('float') / len(s)
    for p in freqs:
        if p != 0.0:
            res -= p * np.log2(p)
    return res

In [8]:
def information_gain(y, x):

    res = entropy(y)

    # We partition x, according to attribute values x_i
    val, counts = np.unique(x, return_counts=True)
    freqs = counts.astype('float') / len(x)

    # We calculate a weighted average of the entropy
    for p, v in zip(freqs, val):
        res -= p * entropy(y[x == v])

    return res

In [9]:
def partition(a):
    return {c: (a == c).nonzero()[0] for c in np.unique(a)}

def is_pure(s):
    return len(set(s)) == 1

In [10]:
def recursive_split(x, y, fields):
    # If there could be no split, just return the original set
    if is_pure(y) or len(y) == 0:
        return y

    # We get attribute that gives the highest information gain
    gain = np.array([information_gain(y, x_attr) for x_attr in x.T])
    selected_attr = np.argmax(gain)

    # If there's no gain at all, nothing has to be done, just return the original set
    if np.all(gain < 1e-6):
        return y

    # We split using the selected attribute
    sets = partition(x[:, selected_attr])

    res = {}
    for k, v in sets.items():
        y_subset = y.take(v, axis=0)
        x_subset = x.take(v, axis=0)

        res["{} = {}".format(fields[selected_attr], k)] = recursive_split(
            x_subset, y_subset, fields)

    return res

In [11]:
tree=recursive_split(x, y, fields)
print(tree)

{'outlook = overcast': array(['yes', 'yes', 'yes', 'yes'], dtype=object), 'outlook = rainy': {'windy = False': array(['yes', 'yes', 'yes'], dtype=object), 'windy = True': array(['no', 'no'], dtype=object)}, 'outlook = sunny': {'humidity = high': array(['no', 'no', 'no'], dtype=object), 'humidity = normal': array(['yes', 'yes'], dtype=object)}}


Decision Trees with C4.5:

In [12]:
def information_gain_ratio(y, x):
    return information_gain(y, x)/entropy(x)

In [75]:
def predict(tree, x , fields):
    for k,i in tree.items():
        #print(k,i)
        find=0
        for j in range(len(x)):
            if k == "{} = {}".format(fields[j], x[j]):
                find=1
                break
        
        if find==0:
            continue
        
        if isinstance(i, dict):
            return predict(i, x, fields)
            break
        else:
            return mode(i)

In [76]:
from statistics import mode
g=predict(tree, x[125,:]  , fields)
print(g)

0


In [59]:
np.unique(y)[1]

1

In [105]:
def get_dict(a):
    for k,i in a.items():
        print(k,i)
        if np.unique(i)!='yes' and np.unique(i)!='no':
            get_dict(i)

In [32]:
dataset = pd.read_csv('titanic_modified.csv')
dataset

,Pclass,Sex,Age,SibSp,Parch,Embarked,Survived
0,3,male,22.0,1,0,S,0
1,1,female,38.0,1,0,C,1
2,3,female,26.0,0,0,S,1
3,1,female,35.0,1,0,S,1
4,3,male,35.0,0,0,S,0
5,3,male,NaN,0,0,Q,0
6,1,male,54.0,0,0,S,0
7,3,male,2.0,3,1,S,0
8,3,female,27.0,0,2,S,1
9,2,female,14.0,1,0,C,1


In [33]:
dataset['Embarked']=dataset['Embarked'].astype(str)

In [34]:
def preprocess_cont_data(data, columns):
    subset=dataset[columns]
    subset =subset.dropna()
    subset= subset.sort_values(by=[columns[0]])
    subset_x=subset.iloc[:, 0].values
    subset_y=subset.iloc[:, 1].values
    return subset_x, subset_y

In [35]:
subset_x, subset_y= preprocess_cont_data(dataset, ['Age', 'Survived'])

In [36]:
x=subset_x
y=subset_y

In [37]:
print(x)

[ 0.42  0.67  0.75  0.75  0.83  0.83  0.92  1.    1.    1.    1.    1.
  1.    1.    2.    2.    2.    2.    2.    2.    2.    2.    2.    2.
  3.    3.    3.    3.    3.    3.    4.    4.    4.    4.    4.    4.
  4.    4.    4.    4.    5.    5.    5.    5.    6.    6.    6.    7.
  7.    7.    8.    8.    8.    8.    9.    9.    9.    9.    9.    9.
  9.    9.   10.   10.   11.   11.   11.   11.   12.   13.   13.   14.
 14.   14.   14.   14.   14.   14.5  15.   15.   15.   15.   15.   16.
 16.   16.   16.   16.   16.   16.   16.   16.   16.   16.   16.   16.
 16.   16.   16.   16.   17.   17.   17.   17.   17.   17.   17.   17.
 17.   17.   17.   17.   17.   18.   18.   18.   18.   18.   18.   18.
 18.   18.   18.   18.   18.   18.   18.   18.   18.   18.   18.   18.
 18.   18.   18.   18.   18.   18.   18.   19.   19.   19.   19.   19.
 19.   19.   19.   19.   19.   19.   19.   19.   19.   19.   19.   19.
 19.   19.   19.   19.   19.   19.   19.   19.   20.   20.   20.   20.
 20.  

In [38]:
def get_threshold(x, y):
    t=[]
    for i in range(len(x)-1):
        t.append((x[i]+x[i+1])/2)
        
    gain=0
    max_gain=0
    t_best=0
    for thresh in t:
        p1=len(x[x<thresh])/len(x)
        p2=len(x[x>thresh])/len(x)
        gain=entropy(y) - p1*entropy(y[x<thresh]) - p2*entropy(y[x>thresh])
        if gain> max_gain:
            max_gain=gain
            t_best=thresh
    return t_best

In [87]:
#F=x.shape[0]/dataset.shape[0]
threshold= get_threshold(x, y)

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [43]:
dataset=dataset.dropna()
dataset

,Pclass,Sex,Age,SibSp,Parch,Embarked,Survived
0,3,male,22.0,1,0,S,0
1,1,female,38.0,1,0,C,1
2,3,female,26.0,0,0,S,1
3,1,female,35.0,1,0,S,1
4,3,male,35.0,0,0,S,0
6,1,male,54.0,0,0,S,0
7,3,male,2.0,3,1,S,0
8,3,female,27.0,0,2,S,1
9,2,female,14.0,1,0,C,1
10,3,female,4.0,1,1,S,1


In [44]:
fields = list(dataset.columns.values[0:6])
fields

['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked']

In [45]:
dataset
x = dataset.iloc[:, :6].values
y = dataset.iloc[:, 6].values


In [46]:
def transform_data(x, threshold):
    for i in range (len(x[:, 2])):
        if x[i, 2] !='nan':
            if x[i, 2]<threshold:
               x[i, 2]=0 
    for i in range (len(x[:, 2])):
        if x[i, 2] !='nan':
            if x[i, 2]>=threshold:
                x[i, 2]=1 
    return x

In [47]:
#x[:,2]=x[:,2].astype(str)
x=transform_data(x,threshold)
x

array([[3, 'male', 1, 1, 0, 'S'],
       [1, 'female', 1, 1, 0, 'C'],
       [3, 'female', 1, 0, 0, 'S'],
       ...,
       [1, 'female', 1, 0, 0, 'S'],
       [1, 'male', 1, 0, 0, 'C'],
       [3, 'male', 1, 0, 0, 'Q']], dtype=object)

In [48]:
def recursive_split_45(x, y, fields):
    # If there could be no split, just return the original set
    if is_pure(y) or len(y) == 10:
        return y

    # We get attribute that gives the highest information gain
    gain = np.array([information_gain_ratio(y, x_attr) for x_attr in x.T])
    selected_attr = np.argmax(gain)

    # If there's no gain at all, nothing has to be done, just return the original set
    if np.all(gain < 1e-6):
        return y

    # We split using the selected attribute
    sets = partition(x[:, selected_attr])

    res = {}
    for k, v in sets.items():
        y_subset = y.take(v, axis=0)
        x_subset = x.take(v, axis=0)

        res["{} = {}".format(fields[selected_attr], k)] = recursive_split(
            x_subset, y_subset, fields)

    return res

In [49]:
tree=recursive_split_45(x, y, fields)
print(tree)

{'Sex = female': {'Pclass = 1': {'Parch = 0': {'Embarked = C': {'SibSp = 0': array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64), 'SibSp = 1': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)}, 'Embarked = Q': array([1], dtype=int64), 'Embarked = S': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1], dtype=int64), 'Embarked = nan': array([1, 1], dtype=int64)}, 'Parch = 1': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64), 'Parch = 2': {'SibSp = 0': array([1, 1, 1, 1, 1], dtype=int64), 'SibSp = 1': array([0, 1, 0, 1], dtype=int64), 'SibSp = 2': array([1, 1], dtype=int64), 'SibSp = 3': array([1, 1], dtype=int64)}}, 'Pclass = 2': {'Parch = 0': {'Embarked = C': array([1, 1, 1, 1, 1], dtype=int64), 'Embarked = Q': array([1], dtype=int64), 'Embarked = S': {'SibSp = 0': {'Age = 0': array([1], dtype=int64), 'Age = 1': array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1], d

In [77]:
g=predict(tree, x[18,:], fields)
print(g)

0


In [572]:
x.shape

(714, 6)

In [86]:
import numpy as np
import pandas as pd
from statistics import mode

# code is based on this resource - http://gabrielelanaro.github.io/blog/2016/03/03/decision-trees.html

def partition(a):
    return {c: (a == c).nonzero()[0] for c in np.unique(a)}


def entropy(s):
    res = 0
    val, counts = np.unique(s, return_counts=True)
    freqs = counts.astype('float') / len(s)
    for p in freqs:
        if p != 0.0:
            res -= p * np.log2(p)
    return res


def information_gain(y, x):

    res = entropy(y)

    # We partition x, according to attribute values x_i
    val, counts = np.unique(x, return_counts=True)
    freqs = counts.astype('float') / len(x)

    # We calculate a weighted average of the entropy
    for p, v in zip(freqs, val):
        res -= p * entropy(y[x == v])

    return res


def is_pure(s):
    return len(set(s)) == 1

def information_gain_ratio(y, x):
    return information_gain(y, x)/entropy(x)


def predict(tree, x , fields):
    for k,i in tree.items():
        #print(k,i)
        find=0
        for j in range(len(x)):
            if k == "{} = {}".format(fields[j], x[j]):
                find=1
                break
        
        if find==0:
            continue
        
        if isinstance(i, dict):
            return predict(i, x, fields)
            break
        else:
            return mode(i)

                
def preprocess_cont_data(data, columns):
    subset=dataset[columns]
    subset =subset.dropna()
    subset= subset.sort_values(by=[columns[0]])
    subset_x=subset.iloc[:, 0].values
    subset_y=subset.iloc[:, 1].values
    return subset_x, subset_y



def get_threshold(x, y):
    t=[]
    for i in range(len(x)-1):
        t.append((x[i]+x[i+1])/2)
        
    gain=0
    max_gain=0
    t_best=0
    for thresh in t:
        p1=len(x[x<thresh])/len(x)
        p2=len(x[x>thresh])/len(x)
        gain=entropy(y) - p1*entropy(y[x<thresh]) - p2*entropy(y[x>thresh])
        if gain> max_gain:
            max_gain=gain
            t_best=thresh
    return t_best



def transform_data(x, threshold):
    for i in range (len(x[:, 2])):
        if x[i, 2] !='nan':
            if x[i, 2]<threshold:
               x[i, 2]=0 
    for i in range (len(x[:, 2])):
        if x[i, 2] !='nan':
            if x[i, 2]>=threshold:
                x[i, 2]=1 
    return x


def recursive_split_45(x, y, fields):
    # If there could be no split, just return the original set
    if is_pure(y) or len(y) == 10: # we set the minimum length of the leaf to 10 (pre-prunning)
        return y

    # We get attribute that gives the highest information gain
    gain = np.array([information_gain_ratio(y, x_attr) for x_attr in x.T]) ## we use the information gain ratio
    selected_attr = np.argmax(gain)

    # If there's no gain at all, nothing has to be done, just return the original set
    if np.all(gain < 1e-6):
        return y

    # We split using the selected attribute
    sets = partition(x[:, selected_attr])

    res = {}
    for k, v in sets.items():
        y_subset = y.take(v, axis=0)
        x_subset = x.take(v, axis=0)

        res["{} = {}".format(fields[selected_attr], k)] = recursive_split(
            x_subset, y_subset, fields)

    return res





pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## we upload titanic dataset
dataset = pd.read_csv('titanic_modified.csv')
dataset['Embarked']=dataset['Embarked'].astype(str)

## we preprocess the real values 'Age'
subset_x, subset_y= preprocess_cont_data(dataset, ['Age', 'Survived'])

# we calculate the threshold for the 'Age' datas
threshold=get_threshold(subset_x, subset_y)

dataset=dataset.dropna()
x = dataset.iloc[:, :6].values
y = dataset.iloc[:, 6].values
fields = list(dataset.columns.values[0:6])

## we transform the continuous features in classified ones
x=transform_data(x,threshold)

## we build the tree and print it 
tree=recursive_split_45(x, y, fields)
print(tree)

## We predict one data from our dataset
i=1
g=predict(tree, x[i,:], fields)
print('the prediction is:',g)






{'Sex = female': {'Pclass = 1': {'Parch = 0': {'Embarked = C': {'SibSp = 0': array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64), 'SibSp = 1': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)}, 'Embarked = Q': array([1], dtype=int64), 'Embarked = S': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1], dtype=int64), 'Embarked = nan': array([1, 1], dtype=int64)}, 'Parch = 1': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64), 'Parch = 2': {'SibSp = 0': array([1, 1, 1, 1, 1], dtype=int64), 'SibSp = 1': array([0, 1, 0, 1], dtype=int64), 'SibSp = 2': array([1, 1], dtype=int64), 'SibSp = 3': array([1, 1], dtype=int64)}}, 'Pclass = 2': {'Parch = 0': {'Embarked = C': array([1, 1, 1, 1, 1], dtype=int64), 'Embarked = Q': array([1], dtype=int64), 'Embarked = S': {'SibSp = 0': {'Age = 0': array([1], dtype=int64), 'Age = 1': array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1], d